In [285]:
import requests
import pandas as pd
from pathlib import Path
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

pd.set_option("display.max_columns", None)

In [286]:
fb = pd.read_csv("../../data/processed/fbref/ENG-Premier League/2024-2025/player_match/misc.csv")

gws = pd.read_csv("../../data/processed/fpl/2024-25/gws/merged_gws.csv")


teams = pd.read_csv("../../data/raw/fpl/2024-25/teams.csv", usecols=["id", "name"])

with open("../../data/processed/_id_lookup_teams.json", "r", encoding="utf-8") as f:
    team_map = json.load(f)
with open("../../data/config/teams.json", "r", encoding="utf-8") as f:
    short_code = json.load(f)

team_map_norm = {k.strip().lower(): v.strip().upper() for k, v in team_map.items()}
short_code_norm = {k.strip().lower(): v.strip().upper() for k, v in short_code.items()}

In [287]:
teams["team"] = teams["name"].str.strip().str.lower().map(short_code_norm)
teams["team_id"] = teams["name"].str.strip().str.lower().map(team_map_norm).str.lower()
teams["opponent_id"] = teams["id"]

#assign the opponents team_id based on the opponent id such that if the opponent_id = 16, the opponent_team_id for that row is the team_id for the team with id=16
id_to_hex = teams.set_index("id")["team_id"].to_dict()   # '39ae6c81' → 'MUN'

#teams = teams.drop(columns="id")

gws = gws[gws["player_id"].notna()]


gws["opponent_id"] = gws["opponent_team"].map(id_to_hex)

code_lookup  = teams.set_index("team_id")["team"]

gws["opponent_team"] = gws["opponent_id"].map(code_lookup)


team_id = teams[["team_id", "opponent_id"]]



In [288]:
team_lookup = teams[["team_id", "team"]]

# 2) merge DataFrame ↔ DataFrame
gws_merged = gws.merge(
    team_lookup,
    on=["team"],      # both columns exist in *both* frames
    how="left"                   # or "inner", as you need
)

display(gws_merged)





,round,first_name,second_name,name,team,position,player_id,fpl_pos,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,modified,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,starts,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,mng_clean_sheets,mng_draw,mng_goals_scored,mng_loss,mng_underdog_draw,mng_underdog_win,mng_win,opponent_id,team_id
0,1,alex,scott,alex scott,BOU,MID,6140a758,MID,1.6,0,0,11,0,12.8,77,0.01,0.01,0.0,1.02,6,1,0,3.6,22.8,2024-08-17T14:00:00Z,62,False,NFO,0,0,0,0,0,4339,1,1,1,0.0,2,0,0,0,50,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ef41d35d,2fdd3467
1,1,carlos,miguel dos santos pereira,carlos miguel,NFO,GK,634e99a6,GK,2.2,0,0,0,0,0.0,427,0.00,0.00,0.0,0.00,6,0,0,0.0,0.0,2024-08-17T14:00:00Z,0,False,BOU,0,0,0,0,0,33324,0,1,1,0.0,0,0,0,0,45,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2fdd3467,ef41d35d
2,1,tomiyasu,takehiro,takehiro tomiyasu,ARS,DEF,38b63d6c,DEF,0.0,0,0,0,0,0.0,22,0.00,0.00,0.0,0.00,2,0,0,0.0,0.0,2024-08-17T14:00:00Z,0,False,WOL,0,0,0,0,0,8462,0,0,2,0.0,0,0,0,0,50,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42ab0080,11cea905
3,1,malcolm,ebiowei,malcolm ebiowei,CRY,MID,c1d71cd0,MID,0.0,0,0,0,0,0.0,197,0.00,0.00,0.0,0.00,8,0,0,0.0,0.0,2024-08-18T13:00:00Z,0,False,BRE,0,0,0,0,0,716,0,1,2,0.0,0,0,0,0,45,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71af9b33,8ace9f04
4,1,ben,brereton díaz,ben brereton,SOU,FWD,00e1d69d,FWD,1.0,0,0,-2,0,14.0,584,0.02,0.32,0.3,0.25,5,1,0,3.3,2.6,2024-08-17T14:00:00Z,70,False,NEW,0,0,0,0,0,66244,1,0,1,16.0,1,0,0,0,55,False,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8a5a1622,aaa42142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26372,9,myles,lewis-skelly,myles lewis-skelly,ARS,DEF,e78bcfe3,DEF,0.3,0,0,5,0,0.4,597,0.00,0.00,0.0,0.31,81,1,0,0.6,3.2,2024-10-27T16:30:00Z,14,False,LIV,0,0,0,0,0,6485,0,2,2,2.0,1,-62,254,316,45,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84d4105e,11cea905
26373,9,giovani,lo celso,giovani lo celso,TOT,MID,004432dc,MID,0.0,0,0,0,0,0.0,493,0.00,0.00,0.0,0.00,86,0,0,0.0,0.0,2024-10-27T14:00:00Z,0,False,CRY,0,0,0,0,0,3592,0,0,1,0.0,0,-118,0,118,49,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8ace9f04,17ac6a5f
26374,9,tawanda,chirewa,tawanda chirewa,WOL,MID,09350915,MID,0.0,0,0,0,0,0.0,538,0.00,0.00,0.0,0.00,84,0,0,0.0,0.0,2024-10-26T14:00:00Z,0,False,BHA,0,0,0,0,0,3939,0,2,2,0.0,0,-187,0,187,45,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80396060,42ab0080
26375,9,ashley,young,ashley young,EVE,DEF,ba08791f,DEF,6.0,1,1,23,0,12.3,238,0.50,0.50,0.0,1.26,87,1,0,4.7,32.4,2024-10-26T16:30:00Z,90,False,FUL,0,0,0,0,0,66981,1,1,1,2.0,6,17991,23660,5669,45,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1aff263c,426079a6


In [289]:
lookup = (
    teams[['team', 'team_id']]          # 'team' is short code (ARS, AVL)
         .drop_duplicates(subset='team')
         .set_index('team')['team_id']  # Series: index = 'ARS', value = '39ae6c81'
)
display(gws_merged)

,round,first_name,second_name,name,team,position,player_id,fpl_pos,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,modified,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,starts,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,mng_clean_sheets,mng_draw,mng_goals_scored,mng_loss,mng_underdog_draw,mng_underdog_win,mng_win,opponent_id,team_id
0,1,alex,scott,alex scott,BOU,MID,6140a758,MID,1.6,0,0,11,0,12.8,77,0.01,0.01,0.0,1.02,6,1,0,3.6,22.8,2024-08-17T14:00:00Z,62,False,NFO,0,0,0,0,0,4339,1,1,1,0.0,2,0,0,0,50,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ef41d35d,2fdd3467
1,1,carlos,miguel dos santos pereira,carlos miguel,NFO,GK,634e99a6,GK,2.2,0,0,0,0,0.0,427,0.00,0.00,0.0,0.00,6,0,0,0.0,0.0,2024-08-17T14:00:00Z,0,False,BOU,0,0,0,0,0,33324,0,1,1,0.0,0,0,0,0,45,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2fdd3467,ef41d35d
2,1,tomiyasu,takehiro,takehiro tomiyasu,ARS,DEF,38b63d6c,DEF,0.0,0,0,0,0,0.0,22,0.00,0.00,0.0,0.00,2,0,0,0.0,0.0,2024-08-17T14:00:00Z,0,False,WOL,0,0,0,0,0,8462,0,0,2,0.0,0,0,0,0,50,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42ab0080,11cea905
3,1,malcolm,ebiowei,malcolm ebiowei,CRY,MID,c1d71cd0,MID,0.0,0,0,0,0,0.0,197,0.00,0.00,0.0,0.00,8,0,0,0.0,0.0,2024-08-18T13:00:00Z,0,False,BRE,0,0,0,0,0,716,0,1,2,0.0,0,0,0,0,45,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71af9b33,8ace9f04
4,1,ben,brereton díaz,ben brereton,SOU,FWD,00e1d69d,FWD,1.0,0,0,-2,0,14.0,584,0.02,0.32,0.3,0.25,5,1,0,3.3,2.6,2024-08-17T14:00:00Z,70,False,NEW,0,0,0,0,0,66244,1,0,1,16.0,1,0,0,0,55,False,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8a5a1622,aaa42142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26372,9,myles,lewis-skelly,myles lewis-skelly,ARS,DEF,e78bcfe3,DEF,0.3,0,0,5,0,0.4,597,0.00,0.00,0.0,0.31,81,1,0,0.6,3.2,2024-10-27T16:30:00Z,14,False,LIV,0,0,0,0,0,6485,0,2,2,2.0,1,-62,254,316,45,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84d4105e,11cea905
26373,9,giovani,lo celso,giovani lo celso,TOT,MID,004432dc,MID,0.0,0,0,0,0,0.0,493,0.00,0.00,0.0,0.00,86,0,0,0.0,0.0,2024-10-27T14:00:00Z,0,False,CRY,0,0,0,0,0,3592,0,0,1,0.0,0,-118,0,118,49,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8ace9f04,17ac6a5f
26374,9,tawanda,chirewa,tawanda chirewa,WOL,MID,09350915,MID,0.0,0,0,0,0,0.0,538,0.00,0.00,0.0,0.00,84,0,0,0.0,0.0,2024-10-26T14:00:00Z,0,False,BHA,0,0,0,0,0,3939,0,2,2,0.0,0,-187,0,187,45,False,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80396060,42ab0080
26375,9,ashley,young,ashley young,EVE,DEF,ba08791f,DEF,6.0,1,1,23,0,12.3,238,0.50,0.50,0.0,1.26,87,1,0,4.7,32.4,2024-10-26T16:30:00Z,90,False,FUL,0,0,0,0,0,66981,1,1,1,2.0,6,17991,23660,5669,45,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1aff263c,426079a6


In [290]:
gws_merged = gws_merged[gws_merged["team_id"].notna()]
na_rows = gws_merged[gws_merged["team_id"].isna()]   # all rows with a NaN

print(gws_merged.columns)


Index(['round', 'first_name', 'second_name', 'name', 'team', 'position',
       'player_id', 'fpl_pos', 'xP', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'modified',
       'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'saves', 'selected', 'starts', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'mng_clean_sheets', 'mng_draw', 'mng_goals_scored', 'mng_loss',
       'mng_underdog_draw', 'mng_underdog_win', 'mng_win', 'opponent_id',
       'team_id'],
      dtype='object')


In [291]:
na_rows = gws_merged[gws_merged["team_id"].isna()]   # all rows with a NaN

In [292]:
gws["game_date"] = pd.to_datetime(gws["kickoff_time"]).dt.date.astype(str)
gws["time"] = pd.to_datetime(gws["kickoff_time"]).dt.time.astype(str)

gws = gws.drop(columns=["mng_clean_sheets", "mng_draw", "mng_goals_scored", "mng_loss", "mng_underdog_draw", "mng_underdog_win", "mng_win"])
display(gws)

,round,first_name,second_name,name,team,position,player_id,fpl_pos,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,modified,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,starts,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,opponent_id,game_date,time
0,1,alex,scott,alex scott,BOU,MID,6140a758,MID,1.6,0,0,11,0,12.8,77,0.01,0.01,0.0,1.02,6,1,0,3.6,22.8,2024-08-17T14:00:00Z,62,False,NFO,0,0,0,0,0,4339,1,1,1,0.0,2,0,0,0,50,False,0,ef41d35d,2024-08-17,14:00:00
1,1,carlos,miguel dos santos pereira,carlos miguel,NFO,GK,634e99a6,GK,2.2,0,0,0,0,0.0,427,0.00,0.00,0.0,0.00,6,0,0,0.0,0.0,2024-08-17T14:00:00Z,0,False,BOU,0,0,0,0,0,33324,0,1,1,0.0,0,0,0,0,45,True,0,2fdd3467,2024-08-17,14:00:00
2,1,tomiyasu,takehiro,takehiro tomiyasu,ARS,DEF,38b63d6c,DEF,0.0,0,0,0,0,0.0,22,0.00,0.00,0.0,0.00,2,0,0,0.0,0.0,2024-08-17T14:00:00Z,0,False,WOL,0,0,0,0,0,8462,0,0,2,0.0,0,0,0,0,50,True,0,42ab0080,2024-08-17,14:00:00
3,1,malcolm,ebiowei,malcolm ebiowei,CRY,MID,c1d71cd0,MID,0.0,0,0,0,0,0.0,197,0.00,0.00,0.0,0.00,8,0,0,0.0,0.0,2024-08-18T13:00:00Z,0,False,BRE,0,0,0,0,0,716,0,1,2,0.0,0,0,0,0,45,False,0,71af9b33,2024-08-18,13:00:00
4,1,ben,brereton díaz,ben brereton,SOU,FWD,00e1d69d,FWD,1.0,0,0,-2,0,14.0,584,0.02,0.32,0.3,0.25,5,1,0,3.3,2.6,2024-08-17T14:00:00Z,70,False,NEW,0,0,0,0,0,66244,1,0,1,16.0,1,0,0,0,55,False,1,8a5a1622,2024-08-17,14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27599,9,myles,lewis-skelly,myles lewis-skelly,ARS,DEF,e78bcfe3,DEF,0.3,0,0,5,0,0.4,597,0.00,0.00,0.0,0.31,81,1,0,0.6,3.2,2024-10-27T16:30:00Z,14,False,LIV,0,0,0,0,0,6485,0,2,2,2.0,1,-62,254,316,45,True,0,84d4105e,2024-10-27,16:30:00
27600,9,giovani,lo celso,giovani lo celso,TOT,MID,004432dc,MID,0.0,0,0,0,0,0.0,493,0.00,0.00,0.0,0.00,86,0,0,0.0,0.0,2024-10-27T14:00:00Z,0,False,CRY,0,0,0,0,0,3592,0,0,1,0.0,0,-118,0,118,49,False,0,8ace9f04,2024-10-27,14:00:00
27601,9,tawanda,chirewa,tawanda chirewa,WOL,MID,09350915,MID,0.0,0,0,0,0,0.0,538,0.00,0.00,0.0,0.00,84,0,0,0.0,0.0,2024-10-26T14:00:00Z,0,False,BHA,0,0,0,0,0,3939,0,2,2,0.0,0,-187,0,187,45,False,0,80396060,2024-10-26,14:00:00
27603,9,ashley,young,ashley young,EVE,DEF,ba08791f,DEF,6.0,1,1,23,0,12.3,238,0.50,0.50,0.0,1.26,87,1,0,4.7,32.4,2024-10-26T16:30:00Z,90,False,FUL,0,0,0,0,0,66981,1,1,1,2.0,6,17991,23660,5669,45,True,0,1aff263c,2024-10-26,16:30:00


In [293]:

# Home = the row where was_home == True
home_rows = gws[gws["was_home"]]
fixture_df = (
    home_rows
      .rename(columns={"team":"home","opponent_team":"away"})
      .loc[:,["game_date","round","home","away"]]
      .drop_duplicates()
)




In [294]:
bridge = (
    fixture_df
      .merge(fb,
             left_on=["home","away"],
             right_on=["is_home","is_away"],
             how="left")
      .loc[:,["game_id","home","away"]]
)

missing = bridge[bridge["game_id"].isna()]

display(bridge)


,game_id,home,away
0,4efc72e4,NFO,BOU
1,4efc72e4,NFO,BOU
2,4efc72e4,NFO,BOU
3,4efc72e4,NFO,BOU
4,4efc72e4,NFO,BOU
...,...,...,...
21788,5e8445c1,BRE,LIV
21789,5e8445c1,BRE,LIV
21790,5e8445c1,BRE,LIV
21791,5e8445c1,BRE,LIV
